In [7]:
# Check GPU Specs
!nvidia-smi

Fri Sep 16 17:01:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# Get hashes to crack
# Make a directory called 'dothashcat' in your Google Drive before this
# with the 'hashes' subdirectory for storing hashes
from google.colab import drive

# This will ask you to go to a link and get an authorization code
# to let the .hashcat folder get synced to your Google Drive
drive.mount('/content/drive')

# Make a symbolic link between Google Drive
# and the local .hashcat directory
!ls /content/drive/My\ Drive/dothashcat
!ln -s /content/drive/My\ Drive/dothashcat /root/.hashcat

Mounted at /content/drive
hashcat.dictstat2  hashcat.potfile  hashes  kernels  sessions


In [9]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  file libmagic-mgc libmagic1
Suggested packages:
  gettext
The following NEW packages will be installed:
  checkinstall file libmagic-mgc libmagic1
0 upgraded, 4 newly installed, 0 to remove and 20 not upgraded.
Need to get 372 kB of arch

In [10]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat --version

v6.2.6-25-g046637d34


In [11]:
# WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat -w 4 -m 1800 --benchmark

hashcat (v6.2.6-25-g046637d34) starting in benchmark mode

nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 11.2)
* Device #1: Tesla T4, 15007/15109 MB, 40MCU

OpenCL API (OpenCL 1.2 CUDA 11.2.109) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla T4, skipped

Benchmark relevant options:
* --workload-profile=4

The plugin 2500 is deprecated and was replaced with plugin 22000. For more details, please read: https://hashcat.net/forum/thread-10253.html

------------------------------------------------------
* Hash-Mode 2500 (WPA-EAPOL-PBKDF2) [Iterations: 4095]
------------------------------------------------------

tcmalloc: large alloc 1405091840 bytes == 0x55d1bdaa8000 @  0x7fb58f208001 0x55d185813c46 0x55d18585cedd 0x55d18580a45c 0x55d18580acbc 0x55d185805aff 0x7fb58e43ac87 0x55d185805b5a
Speed.#1.........:   361.5 kH/s (441.37ms) @ Accel:256 Loops:512 Thr:128 Vec:1

Started: Fri Sep 16 17:04:26 2022
Stopped: Fri Sep 16 17:04:54 2022


In [12]:
# Get wordlists
!mkdir /root/wordlists && cd /root/wordlists

# Clone the SecLists repository
!git clone https://github.com/danielmiessler/SecLists /root/wordlists/SecLists

# Append 'hashcat' for demonstration purposes
!echo "hashcat!" >> /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

!ls /root/wordlists/SecLists

Cloning into '/root/wordlists/SecLists'...
remote: Enumerating objects: 11509, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11509 (delta 4), reused 9 (delta 4), pack-reused 11497
Receiving objects: 100% (11509/11509), 854.56 MiB | 23.08 MiB/s, done.
Resolving deltas: 100% (6241/6241), done.
Checking out files: 100% (5411/5411), done.
CONTRIBUTING.md  Fuzzing  Miscellaneous     Payloads   Web-Shells
CONTRIBUTORS.md  IOCs	  Passwords	    README.md
Discovery	 LICENSE  Pattern-Matching  Usernames


In [13]:
# Check that your hashes are accessible to Colab
!ls /root/.hashcat/hashes

/root/.hashcat/hashes


In [ ]:


# Start pure brute force cracking with a session name
# !hashcat -m 2500 -a3 --session $SESSION /root/.hashcat/hashes/test.hccapx
# !hashcat -m 2500 /root/.hashcat/hashes/test.hccapx -a 3 -w 4 --session test-session

# Restore the session
# !hashcat --session $SESSION --restore

# Crack with a wordlist
!hashcat -m 1800 /root/.hashcat/hashes /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

hashcat (v6.2.6-25-g046637d34) starting

nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 11.2)
* Device #1: Tesla T4, 15007/15109 MB, 40MCU

OpenCL API (OpenCL 1.2 CUDA 11.2.109) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla T4, skipped



In [ ]:
import glob
import subprocess
import re
from tqdm import tqdm
is_cracked = False
wordlists = list()
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Common-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Cracked-Hashes/milw0rm-dictionary.txt'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Default-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Honeypot-Captures/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Leaked-Databases/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Malware/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/WiFi-WPA/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Software/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/*.txt'):
  wordlists.append(wordlist)

In [ ]:
def check_if_cracked(command): 
  output = subprocess.check_output([command + " --show"], shell=True).decode()
  if len(output) == 0:
    return False
  else:
    return True



In [ ]:
cap_file = f"/root/.hashcat/hashes "
for wordlist in tqdm(wordlists):
  command = 'hashcat -m 1800 ' + cap_file + wordlist
  subprocess.call([command],shell=True)
  if check_if_cracked(command):
    is_cracked = True
  if is_cracked:
    print("Password is cracked!!")
    cracked_password = subprocess.check_output([command + " --show"], shell=True).decode()
    cracked_password = cracked_password.split(':')[-1]
    print(f"Password : {cracked_password}")
    print(f"wordlist used : {wordlist}")
    break
  else:
    continue
  
if is_cracked == False:
  print("Password was not found in SecLists")
else:
  pass